# Benchmark test: Cylinder loading test

This notebook contains the test used as benchmark of the elasto-plastic component of this article. This implementation was based on the _[Elasto-plastic analysis of a 2D von Mises material](https://comet-fenics.readthedocs.io/en/latest/demo/2D_plasticity/vonMises_plasticity.py.html)_ demo. Please note that you will need to have install FEniCS 2019.1.0 to run this example. 

First, we import the libraries used in this test, as well as defining compiling options.

In [1]:
import numpy as np
import dolfin as df
import auxiliary_updated as au
from datetime import datetime

'''
Compiler options
'''
df.parameters["form_compiler"]["cpp_optimize"] = True
df.parameters["form_compiler"]["representation"] = 'uflacs'

We import the mesh files used in this test. This meshes were constructed using GMsh, and adapted to FEniCS using [dolfin-convert].

In [2]:
mesh_path = '../meshes/'
file_name = 'cylinder_semispace'
output_files = 'results_cylinder/'

mesh = df.Mesh(mesh_path + file_name + '.xml')
facets = df.MeshFunction('size_t',mesh,mesh_path + file_name+'_facet_region.xml')
volumes = df.MeshFunction('size_t',mesh,mesh_path + file_name+'_physical_region.xml')

We then define the finite element configuration to use in this test, as well as the solution spaces and the mechanical parameters of the benchmark.

In [3]:
'''
Finite elements configuration
'''

ele_us  = df.VectorElement("CG",  mesh.ufl_cell(), 2) # Solid displacement
V = df.VectorFunctionSpace(mesh,'CG', 2) # Solution space for solid displacement
We = df.TensorElement('DG',mesh.ufl_cell(),1,shape=(3,3)) # Tensor element for stress tensor
W_stress = df.FunctionSpace(mesh, We) # Solution space for stress tensor
W0e = df.FiniteElement("DG", mesh.ufl_cell(), 1) # Finite element for cumulative plastic strain
W0 = df.FunctionSpace(mesh, W0e) # Solution space for cumulative plastic strain

'''
Mechanical parameters
'''

E = df.Constant(70e3)
nu = df.Constant(0.3)
K = E/(3*(1 - 2*nu)) # Bulk modulus
G = E/(2*(1 + nu)) # Shear modulus
sig0 = df.Constant(250.0) # Yield strength
Et = E/100. # Tangent modulus
H = E*Et/(E - Et) # Hardening modulus

Then, trial and test functions, as well as internal variables are defined. 

In [4]:
'''
Trial and Test Functions
'''

v = df.TrialFunction(V)
u_ = df.TestFunction(V)

'''
Functions to keep track of the current internal state and increments
'''

sig = df.Function(W_stress)
edev_tensor = df.Function(W_stress, name = 'Deviatoric Strain tensor')
sig_old = df.Function(W_stress)
beta = df.Function(W0, name='Deviatoric Stress reduction')
N = df.Function(W_stress, name='Normal vector to yield surface')
plas_strain = df.Function(W0, name="Cumulative plastic strain")
q_VM = df.Function(W0, name = 'von Mises Equivalent Stress')
p_stress = df.Function(W0, name = 'Mean Stress')
evol_tr = df.Function(W0, name = 'Volumetric strain')
u = df.Function(V, name="Total displacement")
du = df.Function(V, name="Iteration correction")
Du = df.Function(V, name="Current increment")

Boundary conditions, including a radial loading function, are then defined. In our original benchmark, we used 500 time steps. To be able to provide an example that can be easily run outside a cluster, we recommend using 50 time steps.

In [5]:
'''
Boundary conditions
'''
# Bottom
cb1 = df.DirichletBC(V.sub(2), df.Constant(0.0), \
                         facets, 1)

# South
cb2 = df.DirichletBC(V.sub(1), df.Constant(0.0), \
                         facets, 2)

# East
cb3 = df.DirichletBC(V.sub(0), df.Constant(0.0), \
                         facets, 4)
# Top
cb4 = df.DirichletBC(V.sub(2), df.Constant(0.0), \
                         facets, 6)
    
BCS = [cb1,cb2,cb3,cb4]

'''
Normal vector and loading function
'''

n = df.FacetNormal(mesh)
loading = df.Expression('-t', t=0, degree=2)

def F_ext(v):
    return loading*df.dot(n, v)*df.ds(subdomain_data = facets, subdomain_id = 5)

Now, we define functions for the residual and its tangent operator, as well as the plastic strain function.

In [6]:
'''
Residual and TO
'''

TangentOperator = df.inner(au.epsilon(v), au.sigma_tang(K,G,H,u_,beta,N))*df.dx

Residual = -df.inner(au.epsilon(u_),sig)*df.dx + F_ext(u_)

'''
Plastic strain
'''

P0 = df.FunctionSpace(mesh, "DG", 0)
p_avg = df.Function(P0, name="Plastic strain")
S1 = df.TensorFunctionSpace(mesh, 'DG', 0, shape=(3,3))


Finally, we solve the system using a Newton-Raphson scheme. Note that we do not export all the fields than can be generated with our solver. Further studies may modify this output according to its needs.

In [7]:
t = 0.0
con = 0
Nitermax, tol = 200, 1e-8

'''
Newton-Raphson
'''

Nitermax, tol = 200, 1e-8  # parameters of the Newton-Raphson procedure
Nincr = 50
load_steps = np.linspace(0, 500, Nincr+1)[1:]
results = np.zeros((Nincr+1, 2))
con = 0
for i in range(0,np.size(load_steps)):
    loading.t = load_steps[i]
    A, Res = df.assemble_system(TangentOperator, Residual, BCS)
    nRes0 = Res.norm("l2")
    nRes = nRes0
    Du.interpolate(df.Constant((0, 0, 0)))
    niter = 0
    while nRes/nRes0 > tol and niter < Nitermax:
        df.solve(A, du.vector(), Res, "mumps")
        Du.assign(Du+du)
        sig_, N_, beta_, dp_, q_, _, _, _, _ = au.sig_correction(Du,sig_old,plas_strain,K,G,sig0,H)
        au.local_project(sig_, W_stress, df.dx,sig)
        au.local_project(N_, W_stress, df.dx, N)
        au.local_project(beta_, W0, df.dx, beta)
        au.local_project(q_, W0, df.dx, q_VM)
        A, Res = df.assemble_system(TangentOperator, Residual, BCS)
        nRes = Res.norm("l2")
        print('Residual: ',nRes)
        niter += 1
        if niter == Nitermax:
            print('Max number of iterations have been surpassed!')
    u.assign(u+Du)
    sig_old.assign(sig)
    plas_strain.assign(plas_strain+au.local_project(dp_, W0, df.dx))

# Postprocessing

    u.rename('Solid displacement [m]','Desplazamiento del esqueleto sólido')
    plas_strain.rename('Accumulated Plastic Strain','Strain Plastico Acumulado')

    p_avg.assign(df.project(plas_strain, P0))
    
    file = df.File(output_files+"us_test"+str(i)+".pvd")
    file << u
    file = df.File(output_files+"pstrain_test"+str(i)+".pvd")
    file << p_avg    
    file = df.File(output_files+"sigma_test"+str(i)+".pvd")
    sig_proj = df.project(sig,S1)
    sig_proj.rename('Stress Tensor [MPa]','Tensor de stress de Cauchy')
    file << sig_proj
    file = df.File(output_files+"VM_test"+str(i)+".pvd")
    q_proj = df.project(q_VM,P0)
    q_proj.rename('von Mises Equivalent Stress [MPa]','Esfuerzo equivalente de von Mises')
    file << q_proj

    con +=1
    print('Loading steps finished: '+str(con)+'/'+str(Nincr))


Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
           Consider using the option 'quadrature_degree' to reduce the number of points
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
           Consider using the option 'quadrature_degree' to reduce the number of points
Calling FFC just-in-time (JIT) compiler, this may take some time.
           Consider using the option 'quadrature_degree' to reduce the number of points
Residual:  1.9825624147586543e-13
Calling FFC just-in-time (JIT) compiler, this may take some time.
Loading steps finished: 1/50
Residual:  2.02076685394523e-13
Loading steps finished: 2/50
Residual:  2.0131826959932731e-13
Loading steps finished: 3/50
Residual:  2.0228996998144666e-13
Loading steps finished: 4/50
Residual:  2.0304158315358825e-13
Loading steps finished: 5/50
Residual:  2.02582665232051